In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from IPython.display import display 

import spacy

In [3]:
folder = os.path.join(os.getcwd(), "data", "transcripts")
files = [os.path.join(root, f) for root, _, files in os.walk(folder) for f in files if f.endswith(".docx")]
        

In [4]:
from src.pipelinelib.querying import Parser, Queryable

nlp = spacy.load("de_core_news_sm", disable=["ner", "parser"])
parser = Parser(nlp=nlp, metadata_path="./data/transcripts/Kopie von Transkriptionspaare_Daten.xls")

parser.read_from_files(files)

=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 81_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 87_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 182_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 138_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 47_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 105_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 29_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 27_T1_IM_FW.docx ===
=== Parser: reading from /home/rise/Schreibtisch/Sigmund_git/sigmund/data/transcripts/Paar 58_T1_IM_F

In [5]:
display(parser.frame)

,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,1,81,B,M,False,None,Erste Wohnung?
2,0,2,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,3,81,B,M,False,None,Ja.
4,0,4,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...,...
126,9,94,126,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?
127,9,94,127,60,B,M,True,W,Ladet eure Probleme bei mir ab.
128,9,95,128,60,A,W,True,W,Das sind keine Probleme.
129,9,96,129,60,B,M,True,W,Kümmerst du dich drum.


In [6]:
from src.pipelinelib.querying import Parser, Queryable
from src.pipelinelib.text_body import TextBody

queryable = Queryable.from_parser(parser)

In [7]:
document_df = queryable.execute(level=TextBody.DOCUMENT)
display(document_df)

=== Queryable is executing on Document level, query = '' ===


,document_id,couple_id,is_depressed_group,depressed_person,text
0,0,81,False,None,Ich dachte wir reden darüber wie wir die erste...
1,1,87,False,None,"Wie lang?. 10 Minuten. Okay, dann fang mal an...."
2,2,182,False,None,Hm?. Du solltest doch nicht nachgucken.. Sag n...
3,3,138,True,W,"Am Ammersee.. Dann machen wir es so.. Da, wo m..."
4,4,47,True,W,Erzähl mir was.. Also wir müssen die Fragen je...
5,5,105,True,W,Jetzt sollen wir uns unterhalten?. Genau.. Al...
6,6,29,False,None,"Gut, fangen wir mal an.. Ja, fang du mal an.. ..."
7,7,27,True,W,"Positive Gefühle.. Ja, positive Gefühle können..."
8,8,58,False,None,Soll ich anfangen?. Ich hab mir was überlegt.....
9,9,60,True,W,"Wenn´s positiv geht, dann wird es nicht laut, ..."


In [8]:
paragraph_df = queryable.execute(level=TextBody.PARAGRAPH)
display(paragraph_df)

=== Queryable is executing on Paragraph level, query = '' ===


,document_id,paragraph_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,81,B,M,False,None,Erste Wohnung?
2,0,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,81,B,M,False,None,Ja.
4,0,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...
744,9,92,60,B,M,True,W,"Jaaa, klar…"
745,9,93,60,A,W,True,W,Die brauchen neue Reifen.
746,9,94,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?. Ladet eure...
747,9,95,60,A,W,True,W,Das sind keine Probleme.


In [9]:
sentence_df = queryable.execute(level=TextBody.SENTENCE)
display(sentence_df)

queryable.execute(level=TextBody.SENTENCE)

=== Queryable is executing on Sentence level, query = '' ===


,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,1,81,B,M,False,None,Erste Wohnung?
2,0,2,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,3,81,B,M,False,None,Ja.
4,0,4,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...,...
126,9,94,126,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?
127,9,94,127,60,B,M,True,W,Ladet eure Probleme bei mir ab.
128,9,95,128,60,A,W,True,W,Das sind keine Probleme.
129,9,96,129,60,B,M,True,W,Kümmerst du dich drum.


=== Queryable is executing on Sentence level, query = '' ===


,document_id,paragraph_id,sentence_id,couple_id,speaker,gender,is_depressed_group,depressed_person,text
0,0,0,0,81,A,W,False,None,Ich dachte wir reden darüber wie wir die erste...
1,0,1,1,81,B,M,False,None,Erste Wohnung?
2,0,2,2,81,A,W,False,None,"Ja, ich dachte das ist ein nettes Thema, oder?"
3,0,3,3,81,B,M,False,None,Ja.
4,0,4,4,81,A,W,False,None,"Ich fand das süß, wie du erst angefangen hast ..."
...,...,...,...,...,...,...,...,...,...
126,9,94,126,60,B,M,True,W,Hab ich ein Schild auf dem Rücken?
127,9,94,127,60,B,M,True,W,Ladet eure Probleme bei mir ab.
128,9,95,128,60,A,W,True,W,Das sind keine Probleme.
129,9,96,129,60,B,M,True,W,Kümmerst du dich drum.


In [10]:
pgs = queryable.by_couple_id(couple_id=27) \
    .is_depressed(d=False) \
    .execute(level=TextBody.DOCUMENT)
display(pgs)

=== Queryable is executing on Document level, query = '(couple_id == 27) and (is_depressed_group == False)' ===


,document_id,couple_id,is_depressed_group,depressed_person,text


In [ ]:
from src.sigmund.preprocessing.words import Tokenizer, Stemmer, Lemmatizer
from src.sigmund.features.tfidf import FeatureTFIDF
from src.pipelinelib.pipeline import Pipeline
from src.sigmund.classification.naive_bayes import NaiveBayes
from src.sigmund.classification.merger import FeatureMerger
from src.sigmund.extensions import TFIDF_DOCUMENT
from src.sigmund.features.liwc_one_hot import LiwcOneHot

pipeline = Pipeline(queryable=queryable)
pipeline.add_component(Tokenizer())
pipeline.add_component(Stemmer())
pipeline.add_component(Lemmatizer())
pipeline.add_component(FeatureTFIDF())
pipeline.add_component(LiwcOneHot())
pipeline.add_component(FeatureMerger()) #[TFIDF_DOCUMENT]))
pipeline.add_component(NaiveBayes())
display(pipeline.execute())

=== Starting pipeline with ['Tokenizer', 'Stemmer', 'Lemmatizer', 'FeatureTFIDF', 'LiwcOneHot', 'FeatureMerger', 'NaiveBayes'] ===
Executing Tokenizer
=== Queryable is executing on Document level, query = '' ===
=== Queryable is executing on Paragraph level, query = '' ===
=== Queryable is executing on Sentence level, query = '' ===
Executing Stemmer
Executing Lemmatizer
Executing FeatureTFIDF
Executing LiwcOneHot
=== Queryable is executing on Document level, query = '' ===
Executing FeatureMerger
